In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

file_name = '/kaggle/input/marketing/WA_Fn-UseC_-Marketing-Campaign-Eff-UseC_-FastF.csv'
df = pd.read_csv(file_name)

df.head(25)

In [ ]:
print('Raw        : ',df.shape[0])
print('Columns    : ', df.shape[1])
print('\nFeatures   : \n', df.columns.tolist())
print('\nMissing Values : \n', df.isnull().sum())
print('\nUnique Value   : \n', df.nunique())

In [ ]:
df.describe()

EDA

In [ ]:
plt.style.use('ggplot')

ax = df.groupby('Promotion').sum()['SalesInThousands'].plot.pie(figsize = (8,8),
                                                               autopct='%1.0f%%',
                                                               shadow=True,
                                                               explode = (0,0.1,0))

ax.set_ylabel('')
ax.set_title('Sales Distribution Across the 3 Different Promotions')

plt.show()

In [ ]:
# Now we view tge promotions in each market size
df.groupby(['Promotion','MarketSize']).count()['MarketID'].unstack()

In [ ]:
# Put this into plot

ax = df.groupby(['Promotion','MarketSize']).count()['MarketID'].unstack('MarketSize').plot(
kind='bar',
figsize= (12,10),
grid=True)


ax.set_ylabel('Count')
ax.set_title('Breakdowns of market size acriss different promotions')

plt.show()

In [ ]:
# Put this into plot

ax = df.groupby(['Promotion','MarketSize']).count()['MarketID'].unstack('MarketSize').plot(
kind='bar',
figsize= (12,10),
grid=True,
stacked=True)


ax.set_ylabel('Count')
ax.set_title('Breakdowns of market size acriss different promotions')

plt.show()

In [ ]:
df.groupby('AgeOfStore').count()['MarketID']

In [ ]:
ax = df.groupby('AgeOfStore').count()['MarketID'].plot(
kind='bar',
figsize=(12,7),
grid=True)

ax.set_xlabel('Age')
ax.set_ylabel('Count')
ax.set_title('Overall Distributions Store Ages')

plt.show()

In [ ]:
df['MarketID'].value_counts()

In [ ]:
df.groupby(['AgeOfStore','Promotion']).count()['MarketID']

In [ ]:
# Let's plot 

ax = df.groupby(['AgeOfStore','Promotion']).count()['MarketID'].unstack('Promotion').iloc[::-1].plot(
kind='barh',
figsize=(14,18),
grid=True)

ax.set_ylabel('age')
ax.set_xlabel('Count')

ax.set_title('Overall distributions of age of store')

plt.show()

In [ ]:
df.groupby(['Promotion']).describe()['AgeOfStore']

Tabel ini memudahkan untuk memahami distribusi usia toko secara keseluruhan dari statistik ringkas kami.
Kelompok uji memiliki profil usia yang serupa dan rata-rata usia toko adalah sekitar 8 hingga 9 tahun untuk ketiga kelompok ini. 
Sebagian besar toko berusia 10 - 12 tahun atau bahkan lebih muda. Dapat terlihat bahwa profil toko serupa satu sama lain. 
Hal ini menunjukkan bahwa kelompok sampel kami terkendali dengan baik dan hasil pengujian A/B akan bermakna dan dapat dipercaya.

A/B testing 

In [ ]:
means = df.groupby('Promotion')['SalesInThousands'].mean()
stds = df.groupby('Promotion')['SalesInThousands'].std()
ns = df.groupby('Promotion')['SalesInThousands'].count()


T-value:

T-value adalah angka yang mengukur seberapa besar perbedaan antara dua kelompok data. Semakin besar nilainya, semakin besar perbedaannya. Jika nilainya kecil, mungkin perbedaannya juga kecil. T-value memberi tahu kita seberapa kuat perbedaan antara kelompok-kelompok data itu.

P-value:

P-value adalah angka yang memberi tahu kita seberapa mungkin perbedaan yang kita lihat antara dua kelompok data terjadi secara kebetulan. Semakin kecil nilainya, semakin kecil kemungkinan perbedaan itu terjadi karena kebetulan. Jika nilainya sangat kecil, biasanya kita menganggap perbedaan tersebut signifikan secara statistik.

In [ ]:
means

In [ ]:
stds

In [ ]:
ns

Comparing promotion 1 vs promotion 2 in an A/B test

In [ ]:
# Computing the t and p values using scipy

from scipy import stats

t, p = stats.ttest_ind(df.loc[df['Promotion'] == 1, 'SalesInThousands'].values,
                      df.loc[df['Promotion'] == 2, 'SalesInThousands'].values,
                      equal_var=False)

print('t-value = ', t)
print('p-value = ', p) # 0.00000000042903687179871785 

t-value besar, ini menunjukkan bahwa perbedaan antara rata-rata kedua kelompok (promotion 1 dan 2) itu signifikan dan tidak terjadi secara kebetulan.

p yang sangat kecil menunjukkan bahwa perbedaan antara rata-rata kedua kelompok adalah signifikan secara statistik.

Jadi, kombinasi nilai t yang besar dan nilai p yang kecil menunjukkan bahwa perbedaan antara kedua kelompok adalah signifikan



In [ ]:
df.loc[df['Promotion'] == 1, 'SalesInThousands'].values.sum()

In [ ]:
df.loc[df['Promotion'] == 2, 'SalesInThousands'].values.sum()

Comparising promotion 1 vs promotion 3 in an A/B test

In [ ]:
t, p = stats.ttest_ind(
        df.loc[df['Promotion'] == 1, 'SalesInThousands'].values,
        df.loc[df['Promotion'] == 3, 'SalesInThousands'].values,
        equal_var=False)


print('T-Value = ', t)
print('P-value = ', p)

Analisis : 
Nilai T-Value adalah positif, menunjukkan bahwa terdapat perbedaan antara kelompok, meskipun tingkat signifikansinya  tidak terlalu besar.

P-value adalah probabilitas perbedaan antar kelompok terjadi secara kebetulan. Semakin kecil nilai P-value, semakin signifikan perbedaannya. Dalam konteks ini, P-value (0.1205) lebih besar dari tingkat signifikansi yang umumnya digunakan (misalnya 0.05), yang menunjukkan bahwa perbedaan yang diamati mungkin terjadi secara kebetulan.

Dengan demikian, hasil analisis menunjukkan bahwa perbedaan antara kelompok (promotion 1 dan promotion 3) tidak cukup signifikan secara statistik untuk dianggap sebagai hasil yang pasti. Simpelnya dua hal ini sebenarnya sama saja. 

In [ ]:
t, p = stats.ttest_ind(
        df.loc[df['Promotion'] == 2, 'SalesInThousands'].values,
        df.loc[df['Promotion'] == 3, 'SalesInThousands'].values,
        equal_var=False)


print('T-Value = ', t)
print('P-value = ', p)

Dengan demikian, hasil analisis menunjukkan bahwa terdapat perbedaan yang signifikan secara statistik antara dua kelompok dalam sampel, dengan perbedaan tersebut memiliki arah yang berlawanan. Ini menunjukkan bahwa ada faktor atau variabel yang berbeda secara signifikan antara dua kelompok tersebut.

KESIMPULAN : 

Promotion 1 vs Promotion 2:

T-Value yang tinggi (positif) dan P-Value yang sangat rendah menunjukkan bahwa ada perbedaan signifikan antara Promotion 1 dan Promotion 2. Promotion 1 lebih unggul daripada Promotion 2 secara signifikan.

Promotion 1 vs Promotion 3:

T-Value yang rendah dan P-Value yang tinggi menunjukkan bahwa tidak ada perbedaan signifikan antara Promotion 1 dan Promotion 3. Kedua promosi memiliki kinerja yang serupa secara statistik.

Promotion 2 vs Promotion 3:

T-Value yang rendah (negatif) dan P-Value yang sangat rendah menunjukkan bahwa ada perbedaan signifikan antara Promotion 2 dan Promotion 3. Promotion 2 lebih unggul daripada Promotion 3 secara signifikan.

Dengan demikian, berdasarkan hasil A/B testing, Promotion 1 lebih unggul daripada Promotion 2, sementara Promotion 2 lebih unggul daripada Promotion 3.